# **ETL del archivo Homicidios_Hechos**

## **1.Cargando Las Librerías Necesarias**

In [1]:
import pandas as pd #Cargaremos pandas para manejar los datos de nuestros datasets,
import numpy as np #Cargaremos numpy para manejar los datos,
import json as json  #Cargaremos json para manejar los datos de tipo json",
import gzip #Cargaremos gzip para poder abrir archivos con compresion gzip",
import datetime #Cargaremos datetime para manejar fechas",
from google.colab import drive #Cargaremos drive para poder acceder a los archivos de drive"
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
url01="/content/drive/MyDrive/2023/Henry/PI02-DataAnalyst/Datos/homicidios.xlsx"

## **2.Fase ETL**

### 2.1.Carga de Datos

In [3]:
df_homicidios_hechos=pd.read_excel(url01,sheet_name="HECHOS")

In [4]:
df_homicidios_hechos.head()

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


#### 2.2.1.Funciones Para El Analisis Descriptivo De Los Datos

### 3.1.Definiremos funciones para el analisis descriptivo de los datos

In [5]:
def check_df(dataset, head = 5):

  """
  Funcion creada para tener una vista general de la base de datos
  """

  print('*'*30 + 'Forma de la base de datos' + '*'*30, end = '\n'*2)
  print(dataset.shape, end = '\n'*2)#Tamaño de la base de datos

  print('*'*30 + 'Informacion general de la base de datos' + '*'*30, end = '\n'*2)
  print(dataset.info(), end = '\n'*2)# Informacion general

  print('*'*30 + 'NaN cantidad de nulos' + '*'*30, end = '\n'*2)
  print(dataset.isnull().sum(), end = '\n'*2)#Cuenta la cantidad de NaN por columna

#### 2.2.2.Descripcion De Los Datos

In [6]:
check_df(df_homicidios_hechos)

******************************Forma de la base de datos******************************

(696, 21)

******************************Informacion general de la base de datos******************************

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    obj

In [7]:
def Transform_Hora(text):

  '''

  La funcion de Transform_Hora recibe como parametro un dato de cualquier tipo y
  devuelve un objeto de tipo datetime extrayendo la hora del dato.
  Observando 2 formatos de fecha y un tipo de dato string entonces procedemos de
  la siguiente manera :
  format_full='%Y-%m-%d %H:%M:%S'
  format_short='%H:%M:%S'
  definiremos el formato que se presentan y se extraera posteriormente

  '''
  text=str(text)
  format_full='%Y-%m-%d %H:%M:%S'
  format_short='%H:%M:%S'
  if text.count("-")>=2:
    text.replace("1900","2018")
    datetime_object = datetime.datetime.strptime(text, format_full).time()

  elif text.count(":")>=1:
    datetime_object = datetime.datetime.strptime(text, format_short).time()

  else:
    datetime_object="SD"
  return  datetime_object

In [8]:
df_homicidios_hechos['HORA']=df_homicidios_hechos['HORA'].apply(Transform_Hora)

In [29]:
df_homicidios_hechos.head()

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


In [52]:
df_homicidios_hechos1=df_homicidios_hechos.copy()

In [53]:
df_homicidios_hechos1[df_homicidios_hechos1['HORA']=="SD"]

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
518,2019-0103,1,2019-12-18,2019,12,18,SD,SD,"PAZ, GRAL. AV. Y GRIVEO",GRAL PAZ,...,NaN,GRIVEO,"PAZ, GRAL. AV. y GRIVEO",11,Point (94643.11254058 103831.57115061),-58.52169422,-34.59471640,MOTO-MOTO,MOTO,MOTO


In [54]:
df_homicidios_hechos1.head()

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


Observamos que existe aun el dato SD en la columna **HORA** por tanto procedemos a cambiarlo por la moda de la columna en la **comuna** No 11

In [55]:
df_homicidios_hechos1[['HORA','HH']][df_homicidios_hechos1['COMUNA']==11].mode()

,HORA,HH
0,07:30:00,6
1,11:00:00,NaN


In [56]:
df_homicidios_hechos1['HORA']=df_homicidios_hechos1['HORA'].apply(lambda x: datetime.time(7, 30) if x=='SD' else x)

Procederemos a hacer la exploracion en la columna **HH**

In [59]:
df_homicidios_hechos1['HH'].unique()

array([4, 1, 7, 0, 5, 18, 19, 15, 11, 22, 16, 9, 23, 6, 10, 17, 12, 8, 20,
       21, 14, 3, 2, 13, 'SD'], dtype=object)

In [60]:
df_homicidios_hechos1[df_homicidios_hechos1['HH']=='SD']

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
518,2019-0103,1,2019-12-18,2019,12,18,07:30:00,SD,"PAZ, GRAL. AV. Y GRIVEO",GRAL PAZ,...,NaN,GRIVEO,"PAZ, GRAL. AV. y GRIVEO",11,Point (94643.11254058 103831.57115061),-58.52169422,-34.59471640,MOTO-MOTO,MOTO,MOTO


In [64]:
df_homicidios_hechos1['HH']=df_homicidios_hechos1['HH'].apply(lambda x: 7 if x=='SD' else x)

Ahora procederemos a explorar La columna LUGAR_DEL_HECHO

In [70]:
for elemento in df_homicidios_hechos['TIPO_DE_CALLE'].unique():
  print(elemento)

AVENIDA
GRAL PAZ
CALLE
AUTOPISTA


In [69]:
for elemento in df_homicidios_hechos['LUGAR_DEL_HECHO'].unique():
  print(elemento)

AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ
AV GRAL PAZ  Y AV DE LOS CORRALES
AV ENTRE RIOS 2034
AV LARRAZABAL Y GRAL VILLEGAS CONRADO
AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA 
AV 27 DE FEBRERO Y AV ESCALADA
NOGOYA Y JOAQUIN V. GONZALES
AV GENERAL PAZ Y AV DE LOS CORRALES
AV BELGRANO Y BERNARDO DE IRIGOYEN
AV ENTRE RIOS 1366
AV SCALABRINI ORTIZ Y VERA
AV REGIMIENTO DE PATRICIOS Y QUINQUELA MARTIN BENITO
MARIANO ACOSTA Y ANA MARIA JANER
LISANDRO DE LA TORRE Y SEVERO GARCIA GRANDE DE ZEQUEIRA 
SUIPACHA 156
AV DIRECTORIO Y CURAPALIGUE
AV 9 DE JULIO Y ADOLFO ALSINA
AV GRAL PAZ Y AV 27 DE FEBRERO
HOLMBERG Y VEDIA
CURAPALIGUE Y AVELINO DIAZ
VIAMONTE Y AV 9 DE JULIO
AV CABILDO Y DEHEZA
AV GRAL PAZ Y AV CNEL ROCA
AV AMANCIO ALCORTA Y BONAVENA
AV CORRIENTES Y AV CALLAO
HERRERA Y ARISTOBULO DEL VALLE
AV. EVA PERON Y QUIRNO
AV GRAL PAZ COLECTORA Y ZUVIRIA
BARTOLOME MITRE Y AV PUEYRREDON
AV DIRECTORIO Y RIVERA INDARTE
DELLEPIANE LUIS TTE GRAL COLECTORA Y MURGUIONDO
AV JOSE MARIA MORENO Y BALBASTRO
A